In [220]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn import svm
from datetime import date, datetime
from sklearn import tree
import math

In [143]:
data = np.array(pd.read_csv('../data/full_data.csv'))
# data = data[data[:, 9] == 'DEHB001'][:,:9]
data = data[:,:8] # remove the station code and pm2.5

In [157]:
time, x_raw, y_raw = data[:, 0], np.delete(data, [0, 5], axis=1), data[:, [5]]
# Time => Date,
# X => PM10,NO2,NOx,NO,SO2,CO, # PM2.5 (removed in the cell)
# Y => O3

In [158]:
scaler_x = MinMaxScaler()
x = scaler_x.fit_transform(x_raw)
scaler_y = MinMaxScaler()
y = scaler_y.fit_transform(y_raw)[:, 0]

In [159]:
filter_mask = np.logical_and(np.logical_not(np.isnan(y)), np.logical_not(np.isnan(x).any(axis=1)))
# x still contains nan! 
x = x[filter_mask]
y = y[filter_mask]
time = time[filter_mask]

In [160]:
time_year = [int(datetime.fromtimestamp(t).strftime('%Y')) for t in time]
time_month = [int(datetime.fromtimestamp(t).strftime('%m')) for t in time]
time_day = [int(datetime.fromtimestamp(t).strftime('%d')) for t in time]
time_hr = [int(datetime.fromtimestamp(t).strftime('%H')) for t in time]
time_x = np.array([time_year, time_month, time_day, time_hr]).transpose()
# time_x = np.reshape(time_x, (time_x.shape[1], time_x.shape[0]))

In [170]:
x_all = np.concatenate((x, time_x), axis=1)

In [202]:
x_all.shape

(244797, 10)

In [215]:
pca = PCA(n_components=8)
x_low = pca.fit_transform(x_all)
print(x_low)

[[-1.47880344e+01 -1.04801786e+01  5.55742625e+00 ... -2.14969072e-02
  -3.20138947e-03  3.28190512e-02]
 [-1.47871170e+01 -9.48018616e+00  5.55675092e+00 ... -3.93596416e-02
  -3.95093660e-03  9.68902619e-04]
 [-1.47862004e+01 -8.48018757e+00  5.55609225e+00 ... -5.32455304e-02
  -2.50026754e-03 -4.63500738e-04]
 ...
 [ 1.52896980e+01  1.04875774e+01 -5.41566501e+00 ... -1.16764071e-02
  -3.22166834e-03 -1.76318088e-03]
 [ 1.52906120e+01  1.14875924e+01 -5.41628775e+00 ... -7.56528110e-03
  -6.40096069e-03 -1.44526488e-03]
 [-1.47874282e+01 -1.14776025e+01  5.42524625e+00 ... -1.64201837e-02
  -4.58992139e-03 -4.49221544e-03]]


In [216]:
x_train, x_test, y_train, y_test = train_test_split(x_low, y)

In [217]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train, y_train)

In [218]:
clf.score(x_test, y_test)

0.7986715493093973

In [181]:
y_predict = clf.predict(x_test)
score = mean_squared_error(y_predict, y_test)
print(score)

0.0014651222555815016


In [219]:
y_predict_train = clf.predict(x_train)
score = mean_squared_error(y_predict_train, y_train)
print(score)

1.8990846614424616e-09
